In [ ]:
# -*- coding: utf-8 -*-

#########################################################################
############ CARGAR LAS LIBRERÍAS NECESARIAS ############################
#########################################################################

# En caso de necesitar instalar keras en google colab,
# ejecutar la siguiente línea:
# !pip install -q keras
# Importar librerías necesarias
import numpy as np
import keras
import matplotlib.pyplot as plt
import keras.utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
# Optimizador a usar
from keras.optimizers import SGD,Adam

# Importar el conjunto de datos
from keras.datasets import cifar100

INPUT_SHAPE=(32, 32, 3)
weights=None

#########################################################################
######## FUNCIÓN PARA CARGAR Y MODIFICAR EL CONJUNTO DE DATOS ###########
#########################################################################

# A esta función solo se la llama una vez. Devuelve 4 
# vectores conteniendo, por este orden, las imágenes
# de entrenamiento, las clases de las imágenes de
# entrenamiento, las imágenes del conjunto de test y
# las clases del conjunto de test.
def cargarImagenes():
    # Cargamos Cifar100. Cada imagen tiene tamaño
    # (32 , 32, 3). Nos vamos a quedar con las
    # imágenes de 25 de las clases.
    (x_train, y_train), (x_test, y_test) = cifar100.load_data (label_mode ='fine')
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    train_idx = np.isin(y_train, np.arange(25))
    train_idx = np.reshape (train_idx, -1)
    x_train = x_train[train_idx]
    y_train = y_train[train_idx]
    test_idx = np.isin(y_test, np.arange(25))
    test_idx = np.reshape(test_idx, -1)
    x_test = x_test[test_idx]
    y_test = y_test[test_idx]
    
    # Transformamos los vectores de clases en matrices.
    # Cada componente se convierte en un vector de ceros
    # con un uno en la componente correspondiente a la
    # clase a la que pertenece la imagen. Este paso es
    # necesario para la clasificación multiclase en keras.
    y_train = np_utils.to_categorical(y_train, 25)
    y_test = np_utils.to_categorical(y_test, 25)
    
    return x_train , y_train , x_test , y_test

#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve la accuracy de un modelo, 
# definida como el porcentaje de etiquetas bien predichas
# frente al total de etiquetas. Como parámetros es
# necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de
# keras (matrices donde cada etiqueta ocupa una fila,
# con un 1 en la posición de la clase a la que pertenece y un 0 en las demás).
def calcularAccuracy(labels, preds):
    labels = np.argmax(labels, axis = 1)
    preds = np.argmax(preds, axis = 1)
    accuracy = sum(labels == preds)/len(labels)
    return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución
# de la función de pérdida en el conjunto de train y
# en el de validación, y otra con la evolución de la
# accuracy en el conjunto de train y el de validación.
# Es necesario pasarle como parámetro el historial del
# entrenamiento del modelo (lo que devuelven las
# funciones fit() y fit_generator()).
def mostrarEvolucion(hist):
    plt.figure(1)
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['Training loss', 'Validation loss'])
    plt.show()
    
    
    plt.figure(2)
    acc = hist.history['accuracy']
    val_acc = hist.history['val_accuracy']
    plt.plot(acc)
    plt.plot(val_acc)
    plt.legend(['Training accuracy','Validation accuracy'])
    plt.show()

#########################################################################
################## DEFINICIÓN DEL MODELO BASENET ########################
#########################################################################

"""Instancia un modelo básico de baseNet con la arquitectura que se muestra
en la guía de prácticas."""
def basenetBasico():
    #Modelo Sequential inicial
    model = Sequential()
    #Aquitectura siguiendo la guía de prácticas
    model.add(Conv2D(6, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=INPUT_SHAPE))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16,
                     kernel_size = (5, 5),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(50,
                    activation = 'relu'))
    model.add(Dense(25,activation = 'softmax'))
    return model


#########################################################################
######### DEFINICIÓN DEL OPTIMIZADOR Y COMPILACIÓN DEL MODELO ###########
#########################################################################
"""Función para compilar el modelo. De optimizador elegimos SGD."""
def compilar(model):

    # Definimos el optimizador
    opt = SGD(lr = 0.01, decay = 1e-6,
              momentum = 0.9, nesterov = True)

    # Compilamos el modelo
    model.compile(loss = categorical_crossentropy,
                  optimizer = opt,
                  metrics = ['accuracy'])
    return model


# Una vez tenemos el modelo base, y antes de entrenar, vamos a guardar los
# pesos aleatorios con los que empieza la red, para poder reestablecerlos
# después y comparar resultados entre no usar mejoras y sí usarlas.
def guardarPesos(model):
    weights = model.get_weights()
    return weights

#########################################################################
###################### ENTRENAMIENTO DEL MODELO #########################
#########################################################################

"""Función de entrenamiento del modelo model compilado. Si no se le pasa ningún
datagen de ImageDataGenerator, se hace un entrenamiento normal con un
10% de validación. Viene con parámetros por defecto para el tamaño del batch
y las épocas. Devuelve el modelo entrenado, los datos de test (por si
se quiere mostrar el accuray y el loss) y el historial de entrenamiento"""
def entrenarModelo(model, datagen="foo", batch_size=32, epocas=12):
    #Guardamos los pesos por si reutilizamos el modelo
    guardarPesos(model)
    #Cargamos las imágenes de train y test
    x_train , y_train , x_test , y_test=cargarImagenes()
    if(datagen=="foo"):
        #Entrenamos el modelo de forma normal si no hay datagen
        historial = model.fit(x_train, y_train,
                              validation_split=0.1,
                              batch_size=batch_size,epochs=epocas,verbose=1)
    else:
        #Lista de callbacks para el early stopping
        cb_list = []

        # Paramos si no mejoramos en un número determinado de épocas
        es_loss = EarlyStopping(monitor = 'val_loss',
                                            patience = 15,
                                            restore_best_weights = True)
        es_acc = EarlyStopping(monitor = 'val_accuracy',
                                          patience = 15,
                                          restore_best_weights = True)

        cb_list.append(es_loss)
        cb_list.append(es_acc)
        #Entrenamos modelo con el datagen dado por parámetro
        #Primero aplicamos el preprocesamiento
        datagen.fit(x_train)
        #Misma normalización a los datos de test
        datagen.standardize(x_test)
        #Generamos el entrenamiento para fit_generator
        generator_train=datagen.flow(x_train, y_train,
                     batch_size = batch_size, subset ='training')   
        #Generamos la validación para fit_generator
        generator_valid=datagen.flow(x_train, y_train,
                     batch_size = batch_size, subset ='validation')
        #Entrenamos el modelo
        historial=model.fit_generator(generator_train,
                            steps_per_epoch = len(x_train)*0.9/32,
                            epochs = epocas, 
                            validation_data = generator_valid, 
                            validation_steps = len(x_train)*0.1/32,
                            callbacks=cb_list)
        #Mostramos información del early stopping y el entrenamiento
        mejor_epoc = len(historial.epoch) - 15
        print("\nNo se ha mejorado en las últimas 15 épocas.")
        print("Mejores pesos obtenidos en la época", mejor_epoc)
    #Devolvemos el modelos y el test para calcular el accuracy
    #Devolvemos también el historial de entrenamiento
    return model, x_test, y_test, historial

#########################################################################
################ PREDICCIÓN SOBRE EL CONJUNTO DE TEST ###################
#########################################################################

"""Muestra el accuracy y loss del modelo"""
def prediccionTest(model, x_test, y_test):
    #Evaluamos el modelo
    score = model.evaluate(x_test, y_test, verbose=0)
    print("BaseNet evaluación")
    #Mostramos la accuracy
    print('Test accuracy:', score[1])
    #Mostramos la pérdida
    print('Test loss:', score[0])

#########################################################################
########################## MEJORA DEL MODELO ############################
#########################################################################

"""Versión mejorada de la arquitectura de modelo BaseNet pedida en la guía
de prácticas."""
def basenetMejorado():
    #Modelo Sequential inicial
    model = Sequential()
    #Hacer una convolución de tamaño 5 es lo mismo que hacer dos de 3 así
    #que sustituimos por dos de 2 obteniendo así más profundidad
    model.add(Conv2D(32,
                     padding = 'same',
                     kernel_size = (3, 3),
                     input_shape = INPUT_SHAPE))
    #batchNormalization después de cada convolución totalmente conectada
    #(menos la última) para mantener la media más o menos a 1 y varianza 0
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32,
                     kernel_size = (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64,
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    #Añado una convolución más para aumentar la profundidad con padding para
    #no cambiar la dimensión de salida
    model.add(Conv2D(128,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(50))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(25,activation = 'softmax'))
    return model


#########################################################################
########## Funciones para las ejecuciones de los apartados ##############
#########################################################################

"""Ejecución del apartado 1. Instanciamos el modelo y lo entrenamos.
Después mostramos su evolución."""
def apartado1():
    print("Apartado 1")
    #Instanciamos el modelo báscico de BaseNet
    modelo=basenetBasico()
    #Compilamos el modelo antes de entrenarlo
    modelo=compilar(modelo)
    #Entrenamiento del modelo
    modelo, xt, yt, historial=entrenarModelo(modelo, epocas=30)
    #Función para mostrar la accuracy
    prediccionTest(modelo,xt, yt)
    #Mostramos la evolución del modelo en el entrenamiento
    mostrarEvolucion(historial)

"""Ejecución del apartado 2. Instanciamos el modelo y el datagen y lo entrenamos.
Después mostramos su evolución."""
def apartado2():
    input("\n--- Pulsar tecla para continuar. Ejecucion apartado 2 ---\n")
    #Instanciamos el modelo báscico de BaseNet
    modelo=basenetMejorado()
    #Compilamos el modelo antes de entrenarlo
    modelo=compilar(modelo)
    datagen=ImageDataGenerator(featurewise_center = True,
                              featurewise_std_normalization = True,
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              validation_split = 0.1)
    #Entrenamiento del modelo
    modelo, xt, yt, historial=entrenarModelo(modelo,datagen=datagen,epocas=100)
    #Función para mostrar la accuracy
    prediccionTest(modelo,xt, yt)
    #Mostramos la evolución del modelo en el entrenamiento
    mostrarEvolucion(historial)
###################
"""Bonus"""
###################
"""Versión mejorada de la arquitectura de modelo BaseNet del apartado 2."""
def basenetBonus():
    #Modelo Sequential inicial
    model = Sequential()
    #Hacer una convolución de tamaño 5 es lo mismo que hacer dos de 3 así
    #que sustituimos por dos de 2 obteniendo así más profundidad
    model.add(Conv2D(32,
                     padding = 'same',
                     kernel_size = (3, 3),
                     input_shape = INPUT_SHAPE))
    #batchNormalization después de cada convolución totalmente conectada
    #(menos la última) para mantener la media más o menos a 1 y varianza 0
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32,
                     kernel_size = (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64,
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    #Añado una convolución más para aumentar la profundidad con padding para
    #no cambiar la dimensión de salida
    model.add(Conv2D(128,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Conv2D(256,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256,
                     padding = 'same',
                     kernel_size = (3, 3)))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(200))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(50))
    #batchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(25,activation = 'softmax'))
    return model

"""Ejecución del bonus. Instanciamos el modelo y el datagen y lo entrenamos.
Después mostramos su evolución. Intentamos mejorar el modelo obtenido en el 
apartado 2. Para ello añadimos cizallamiento en el datagen y la posibilidad
de rotar imágenes."""
def bonus():
    input("\n--- Pulsar tecla para continuar. Ejecucion bonus ---\n")
    #Instanciamos el modelo báscico de BaseNet
    modelo=basenetBonus()
    #Compilamos el modelo antes de entrenarlo
    modelo=compilar(modelo)
    datagen=ImageDataGenerator(featurewise_center = True,
                              featurewise_std_normalization = True,
                              rotation_range=40,
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              shear_range=0.2,
                              validation_split = 0.1)
    #Entrenamiento del modelo
    modelo, xt, yt, historial=entrenarModelo(modelo,datagen=datagen,epocas=150)
    #Función para mostrar la accuracy
    prediccionTest(modelo,xt, yt)
    #Mostramos la evolución del modelo en el entrenamiento
    mostrarEvolucion(historial)
#########################################################################
####################### Ejecuciones apartados ###########################
#########################################################################


apartado1()
apartado2()
bonus()